In [ ]:
import tensorflow as tf
import glob as gb
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import imgaug as ia
import imgaug.augmenters as iaa
import tensorflow.keras.layers as tfl

In [ ]:
!pip install kaggle

In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = '/content'

In [ ]:
!kaggle datasets download -d kumaresanmanickavelu/lyft-udacity-challenge

In [ ]:
!unzip lyft-udacity-challenge.zip

In [ ]:
classes = ['dataB', 'dataD', 'dataA', 'dataC', 'dataE']

In [ ]:
img_path =   [f"{os.path.join('/content/', i, i, 'CameraRGB')}/{x}" for i in classes for x in os.listdir(os.path.join('/content/', i, i, 'CameraRGB')) if not x.startswith('.')]
masks_path = [f"{os.path.join('/content/', i, i, 'CameraSeg')}/{x}" for i in classes for x in os.listdir(os.path.join('/content/', i, i, 'CameraSeg')) if not x.startswith('.')]

In [ ]:
nub = 5


f, axis = plt.subplots(nub,2, figsize=(20,20))

for i,x in enumerate(np.random.random_integers(0,5000,nub)):
    images = cv2.imread(img_path[x])
    masks = cv2.imread(masks_path[x])
    masks = tf.math.reduce_max(masks, axis=-1, keepdims=True)
    axis[i][0].imshow(images)
    axis[i][1].imshow(masks)

plt.show()


In [ ]:
imshape = (256, 256, 3)
numclasses = 13



ia.seed(1)

seq = iaa.Sequential([
    # apply the following augmenters to most images
    #iaa.Rotate(rotate=(-90,90)),
    # iaa.Affine(rotate=(-90,90)),
    iaa.Crop(px=(0, 16)),
    iaa.Fliplr(0.5),
    iaa.Multiply(mul=(0.6, 1.3)),
    # iaa.Sometimes(0.5, iaa.GaussianBlur(sigma=(0,6)))
    ],  random_order=True)




class DataGen(tf.keras.utils.Sequence):

    def __init__(self,image_paths, annot_paths, batch_size=32, shuffle=True, augment=False):

        self.image_paths = image_paths
        self.annot_paths = annot_paths
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.augment = augment
        self.on_epoch_end()


    def __len__(self):
        # Denotes the number of batches per epoch
        return int(np.floor(len(self.image_paths) / self.batch_size))


    def __getitem__(self, index):
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        image_paths = [self.image_paths[k] for k in indexes]
        annot_paths = [self.annot_paths[k] for k in indexes]

        X, y = self.__data_generation(image_paths, annot_paths)

        return X, y


    def on_epoch_end(self):
        # Updates indexes after each epoch
        self.indexes = np.arange(len(self.image_paths))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)





    def __data_generation(self, image_paths, annot_paths):
        X = np.empty((self.batch_size, imshape[0],imshape[1],imshape[2]), dtype= np.float32)
        Y = np.empty((self.batch_size, imshape[0],imshape[1],numclasses), dtype= np.float32)
        for i, (im_path, an_path) in enumerate(zip(image_paths, annot_paths)):
            img = cv2.imread(im_path,1)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            mask = cv2.imread(an_path)
            img = cv2.resize(img, (imshape[1], imshape[0]))
            mask = cv2.resize(mask, (imshape[1], imshape[0]))
            mask = tf.math.reduce_max(mask, axis=-1)
            mask = tf.one_hot(mask, depth=13, dtype=tf.uint8)
            mask = np.array(mask)



            #Data Augmentation
            # aug_det = seq.to_deterministic()
            # img = aug_det.augment_image(img)
            # mask = aug_det.augment_image(mask)

            X[i,] = img.astype(np.float32)/255
            Y[i,] = mask
        return X,Y







dg = DataGen(img_path, masks_path, batch_size=16)

In [ ]:
x,y = dg.__getitem__(1)

In [ ]:
np.unique(y)

In [ ]:
plt.imshow(x[1])

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import backend as K

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

def conv_block(inputs, filters, kernel_size=3, activation='relu', padding='same'):
    x = Conv2D(filters, kernel_size, activation=activation, padding=padding)(inputs)
    x = BatchNormalization()(x)
    x = Conv2D(filters, kernel_size, activation=activation, padding=padding)(x)
    x = BatchNormalization()(x)
    return x

def unet_model(input_shape, num_classes):
    inputs = tf.keras.Input(shape=input_shape)

    # Encoder
    conv1 = conv_block(inputs, 64)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = conv_block(pool1, 128)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = conv_block(pool2, 256)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = conv_block(pool3, 512)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    # Bottleneck
    conv5 = conv_block(pool4, 1024)

    # Decoder
    up1 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(conv5)
    up1 = concatenate([up1, conv4], axis=-1)
    up1 = conv_block(up1, 512)

    up2 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(up1)
    up2 = concatenate([up2, conv3], axis=-1)
    up2 = conv_block(up2, 256)

    up3 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(up2)
    up3 = concatenate([up3, conv2], axis=-1)
    up3 = conv_block(up3, 128)

    up4 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(up3)
    up4 = concatenate([up4, conv1], axis=-1)
    up4 = conv_block(up4, 64)

    # Output
    outputs = Conv2D(num_classes, 1, activation='softmax')(up4)

    model = Model(inputs, outputs)
    return model

patch_size = (256, 256)
stride = 128

input_shape = (256, 256, 3)  # Adjust based on your input data shape (e.g., RGB images)
num_classes = 13  # Adjust based on the number of segmentation classes in your dataset



In [ ]:
#reate the U-Net model
model = unet_model(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
def dice(y_true, y_pred, smooth=1.):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coefficient(y_true, y_pred, smooth=1):
    intersection = K.sum(y_true * y_pred)
    union = K.sum(y_true) + K.sum(y_pred)
    dice = (2.0 * intersection + smooth) / (union + smooth)
    return dice






In [ ]:
model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=[ 'accuracy',dice_coefficient])

checkpoint = ModelCheckpoint(os.path.join( 'unet.h5'), monitor='dice', verbose=1, mode='max',
                             save_best_only=True, save_weights_only=True)


model.fit_generator(generator=dg,
                    steps_per_epoch=len(dg),
                    epochs=500, verbose=1,
                    callbacks=[checkpoint])